# Урок 4. Парсинг HTML. XPath

Написать приложение, которое собирает основные новости с сайта на выбор news.mail.ru, lenta.ru, yandex-новости. 

Для парсинга использовать XPath. 

Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.

### Get news data

In [12]:
from lxml import html
import requests

In [13]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
}

url = 'https://lenta.ru/'

response = requests.get(url, headers=headers)

In [14]:
dom = html.fromstring(response.text)

news = dom.xpath('//div[@class="span8 js-main__content"]//div[@class="item"]')
news_list = []

for item in news:
  item_data = {}

  name = item.xpath('.//a/text()')
  link = item.xpath('.//a/@href')
  time = item.xpath('.//a/time/@datetime')

  item_data['source'] = 'https://lenta.ru/'
  item_data['name'] = name[0]
  item_data['link'] = link[0]
  item_data['time'] = time[0]

  news_list.append(item_data)

In [15]:
import pandas as pd
df = pd.DataFrame(news_list)
df

,source,name,link,time
0,https://lenta.ru/,Смертельная перестрелка произошла в британском...,/news/2021/08/12/plimuth/,"23:31, 12 августа 2021"
1,https://lenta.ru/,На Украине прокомментировали заявление России ...,/news/2021/08/12/nikolenko/,"23:25, 12 августа 2021"
2,https://lenta.ru/,Очевидцы взрыва автобуса в Воронеже рассказали...,/news/2021/08/12/ochevidcy/,"23:09, 12 августа 2021"
3,https://lenta.ru/,Появилось видео момента взрыва автобуса в Воро...,/news/2021/08/12/vzr_vid/,"23:04, 12 августа 2021"
4,https://lenta.ru/,Названа возможная зарплата самого дорогого игр...,/news/2021/08/12/footmoney/,"22:56, 12 августа 2021"
5,https://lenta.ru/,Водитель взорвавшегося в Воронеже автобуса рас...,/news/2021/08/12/voditell/,"22:46, 12 августа 2021"
6,https://lenta.ru/,Названа возможная причина взрыва автобуса в Во...,/news/2021/08/12/avtobus/,"22:28, 12 августа 2021"
7,https://lenta.ru/,У коронавируса обнаружили «ахиллесову пяту»,/news/2021/08/12/achilles/,"22:08, 12 августа 2021"
8,https://lenta.ru/,В Воронеже взорвался автобус,/news/2021/08/12/auto/,"21:49, 12 августа 2021"


### Сложить собранные данные в БД

In [16]:
! pip3 install pymongo[srv]

In [17]:
from pymongo import MongoClient

In [18]:
usr = 'db_user'
pwd = 'mOS9LygLPkiXsliW'
dbname = 'ds_course'

url = 'mongodb+srv://' + usr + ':' + pwd + '@myfreecluster.sns97.mongodb.net/' + dbname + '?retryWrites=true&w=majority'

client = MongoClient(url)

In [19]:
db = client[dbname]
news_collection = db.news

In [20]:
if news_collection.count_documents({}) == 0:
  jobs_list_collection.insert_many(news_list)
else:
  for item in news_list:
    query = {
        'link': item['link']
    }

    update = {
        '$set': item
    }

    news_collection.update_one(query, update, upsert=True)